In [ ]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
with open('config.json', 'r') as f:
    conf = json.load(f)
vals = list(conf['cols']['values'].keys())
meta = conf['cols']['meta']
cols = vals + meta

In [ ]:
# helpers
get_avg = lambda w, v: np.sum((w * v.T).T, axis=0) / np.sum(w)
get_var = lambda w, v: np.sum((w * ((v - get_means(w, v)) ** 2).T).T, axis=0) / np.sum(w)

def z_score(Y, C):
    avg_avg = np.zeros(21)
    avg_var = np.zeros(21)
    for c in C:
        avg_avg += list(Y[c]['avg'].values())[0] / len(C)
        avg_var += list(Y[c]['var'].values())[0] / len(C)
    
    var_avg = np.zeros(21)
    var_var = np.zeros(21)
    for c in C:
        var_avg += ((list(Y[c]['avg'].values())[0] - avg_avg) ** 2) / len(C)
        var_var += ((list(Y[c]['var'].values())[0] - avg_var) ** 2) / len(C)

        
    for k1, v1 in Y.items():
        for k2, v2 in v1['avg'].items():
            Y[k1]['avg'][k2] = ((v2 - avg_avg) / var_avg ** .5).tolist()
        for k2, v2 in v1['var'].items():
            Y[k1]['var'][k2] = ((v2 - avg_var) / var_var ** .5).tolist()
    return Y

In [ ]:
ess          = pd.read_csv('data/ess/ESS8e02_2-ESS9e03_1-ESS10.csv', dtype='object', usecols=cols)
ess          = ess[ess['dweight'].notna()]
ess['cntry'] = ess['cntry'].apply(lambda x: x.lower())
groups       = ess.groupby(['cntry', 'essround']).groups


In [ ]:
Y = {cntry : {'avg': {}, 'var' : {}} for cntry in list(set(ess['cntry']))}
for k, v in groups.items():
    group   = ess.iloc[v]
    values  = (group[vals].to_numpy()).astype(np.int64)
    weights = (group['dweight'].to_numpy()).astype(np.float64)
    avg     = get_avg(weights, values) # weigthed means for country round
    var     = get_var(weights, values)
    Y[k[0]]['avg'][k[1]] = avg
    Y[k[0]]['var'][k[1]] = var
Y = z_score(Y, list(set(ess['cntry']))[:7])    

In [ ]:
with open('data/ess/ess.json', 'w') as f:
    json.dump(Y, f)